In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
%matplotlib qt
plt.style.use('seaborn')
plt.style.use('seaborn-colorblind')

C:\Users\plyslab\AppData\Local\Temp\ipykernel_14748\3776523248.py:2: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')
C:\Users\plyslab\AppData\Local\Temp\ipykernel_14748\3776523248.py:3: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-colorblind')


### Sweeping across the first frequency while changing centre-point

In [3]:
from experiment import Experiment
E = Experiment()

Welcome to the FMR Experiment!
You might want to update some experiment parameters.
The parameters presently are:

PS Output Current (A) :	 3.16894
PS Output Voltage (V) :	 47.5699
PS Output Mode (Current/Voltage) :	 Constant Current
SG Frequency :	 FR03000000000HZ
SG RF Output :	 On
SG RF Output Level :	 LE-70.0DM
LIA Sensitivity :	 0.0005
LIA Time Constant :	 0.3
Read Delay (s) :	 1
Log File :	 c:\Users\plyslab\Desktop\SpinLab_FMR_Automation\Experiment_Logs\FMR_log_2023-6-7_17-57-31.log


In [4]:
# Changing any parameters we need to change
E.SG.level = '-3'

# Creating some constants to help us later
save_dir = r'C:\Users\plyslab\Desktop\different_ranges'

In [5]:
# Let's do one sweep @ 3 GHz
frequency = 3
field_range = np.linspace(30, 200, 200)

E.sweep_field(3, field_range, save_dir=save_dir, savefig=True)

c:\Users\plyslab\Desktop\SpinLab_FMR_Automation\experiment.py:211: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  self.ax.set_xlim(min(xdata), max(xdata))


In [8]:
# Looks good. Now let's do a sweep at 3.5 GHz
frequency = 3.5
field_range = np.linspace(50, 230, 200)

E.sweep_field(frequency, field_range, save_dir=save_dir, savefig=True)

c:\Users\plyslab\Desktop\SpinLab_FMR_Automation\experiment.py:211: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  self.ax.set_xlim(min(xdata), max(xdata))


**Take some time to pause**

As expected, the centre-point does go ahead a bit. We'll use our past two measurements as a guideline for how much the field sweep range should change as we modify the frequency.

Let's see the exact centre-points for our past two measurements.

In [6]:
csv_path = save_dir + r'\freq_3_GHz_field_30.0–200.0_Oe.csv'
df_3ghz = pd.read_csv(csv_path)

# Get the midpoint from the 3GHz file
field_at_minX_3ghz = df_3ghz.iloc[df_3ghz['X'].idxmin()]['field_Oe']
field_at_maxX_3ghz = df_3ghz.iloc[df_3ghz['X'].idxmax()]['field_Oe']
field_at_minY_3ghz = df_3ghz.iloc[df_3ghz['Y'].idxmin()]['field_Oe']
field_at_maxY_3ghz = df_3ghz.iloc[df_3ghz['Y'].idxmax()]['field_Oe']

midpoint_X_3ghz = (field_at_minX_3ghz + field_at_maxX_3ghz) / 2
midpoint_Y_3ghz = (field_at_minY_3ghz + field_at_maxY_3ghz) / 2

midpoint_3ghz = (midpoint_X_3ghz + midpoint_Y_3ghz) / 2

print('X field Midpoint (Oe) @ 3GHz:' + '\t', midpoint_X_3ghz)
print('Y field Midpoint (Oe) @ 3GHz:' + '\t', midpoint_Y_3ghz)
print('Average field Midpoint (Oe) @ 3GHz:' + '\t', midpoint_3ghz)

X field Midpoint (Oe) @ 3GHz:	 106.88442211055275
Y field Midpoint (Oe) @ 3GHz:	 106.03015075376885
Average field Midpoint (Oe) @ 3GHz:	 106.4572864321608


Great! Now let's find out the centre-ponts at 3.5 GHz. But all that above is quite lengthy, and since we're going to be finding midpoints a lot, let's use a helper function (which does the same thing). We've made one in the `experiment.py` file. Just always double check your midpoints, as noise in data can change them easily.

In [7]:
# Get the midpoint from the 3.5GHz file
from experiment import get_midpoint

csv_path = save_dir + r'\freq_3.5_GHz_field_50.0–230.0_Oe.csv'

midpoint_X_3_5ghz = get_midpoint(csv_path, channel='X')
midpoint_Y_3_5ghz = get_midpoint(csv_path, channel='Y')
midpoint_3_5ghz = get_midpoint(csv_path, channel='both')

print('\nX field Midpoint (Oe) @ 3.5GHz:' + '\t', midpoint_X_3_5ghz)
print('Y field Midpoint (Oe) @ 3.5GHz:' + '\t', midpoint_Y_3_5ghz)
print('Average field Midpoint (Oe) @ 3.5GHz:' + '\t', midpoint_3_5ghz)


X field Midpoint (Oe) @ 3.5GHz:	 150.85427135678393
Y field Midpoint (Oe) @ 3.5GHz:	 149.9497487437186
Average field Midpoint (Oe) @ 3.5GHz:	 150.40201005025125


Great! Now we have a pretty good idea of where the midpoints of the two field sweep curves lie. Let's see how much of a difference we get.

In [8]:
print('Midpoint difference between 3GHz and 3.5GHz is', (midpoint_3_5ghz - midpoint_3ghz), 'Oe')

Midpoint difference between 3GHz and 3.5GHz is 43.94472361809045 Oe


That's a 43.9 Oe increase in field centrepoint per 0.5 GHz increase in frequency.

Now we expect the relationship between field midpoint and frequency to be approximately linear, but that might not be a good enough approximation depending on the sample. Let's see if this linear approximation is good enough. We will check the midpoint difference between sweep with frequencies 3 GHz and 3.5 GHz.

In [9]:
# Sweep at 5 GHz
frequency = 5.0
field_range = np.linspace(230, 400, 200) # Field range approximation from our 50Oe / 0.5 GHz figure.

E.sweep_field(frequency, field_range, save_dir=save_dir, savefig=True)

c:\Users\plyslab\Desktop\SpinLab_FMR_Automation\experiment.py:215: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  self.ax.set_xlim(min(xdata), max(xdata))


In [10]:
# Sweep at 5.5 GHz
frequency = 5.5
field_range = np.linspace(280, 450, 200) # Field range approximation from our 50Oe / 0.5 GHz figure.

E.sweep_field(frequency, field_range, save_dir=save_dir, savefig=True)

c:\Users\plyslab\Desktop\SpinLab_FMR_Automation\experiment.py:215: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  self.ax.set_xlim(min(xdata), max(xdata))


All looks good so far. Let's see if the field midpoints between 5 GHz and 5.5 GHz have same difference.

In [12]:
# Get the midpoint from the 5 GHz file
csv_path = save_dir + r'\freq_5.0_GHz_field_230.0-400.0_Oe.csv'

midpoint_X_5ghz = get_midpoint(csv_path, channel='X')
midpoint_Y_5ghz = get_midpoint(csv_path, channel='Y')
midpoint_5ghz = get_midpoint(csv_path, channel='both')

print('\nX field Midpoint (Oe) @ 5GHz:' + '\t', midpoint_X_5ghz)
print('Y field Midpoint (Oe) @ 5GHz:' + '\t', midpoint_Y_5ghz)
print('Average field Midpoint (Oe) @ 5GHz:' + '\t', midpoint_5ghz)


X field Midpoint (Oe) @ 5GHz:	 307.3115577889447
Y field Midpoint (Oe) @ 5GHz:	 306.4572864321608
Average field Midpoint (Oe) @ 5GHz:	 306.88442211055275


In [13]:
# Get the midpoint from the 5 GHz file
csv_path = save_dir + r'\freq_5.5_GHz_field_280.0-450.0_Oe.csv'

midpoint_X_5_5ghz = get_midpoint(csv_path, channel='X')
midpoint_Y_5_5ghz = get_midpoint(csv_path, channel='Y')
midpoint_5_5ghz = get_midpoint(csv_path, channel='both')

print('\nX field Midpoint (Oe) @ 5.5GHz:' + '\t', midpoint_X_5_5ghz)
print('Y field Midpoint (Oe) @ 5.5GHz:' + '\t', midpoint_Y_5_5ghz)
print('Average field Midpoint (Oe) @ 5.5GHz:' + '\t', midpoint_5_5ghz)


X field Midpoint (Oe) @ 5.5GHz:	 369.69849246231155
Y field Midpoint (Oe) @ 5.5GHz:	 370.97989949748745
Average field Midpoint (Oe) @ 5.5GHz:	 370.3391959798995


In [14]:
print('Midpoint difference between 5GHz and 5.5GHz is', (midpoint_5_5ghz - midpoint_5ghz), 'Oe')

Midpoint difference between 5GHz and 5.5GHz is 63.45477386934675 Oe


So there is a significant (ish) difference. 43 Oe / 0.5 GHz rate of change of field midpoint has changed to 63.5 Oe / 0.5 GHz. That means we can expect a drift of 100 Oe for a 10Ghz change in frequency.

We can deal with this by just increasing our field range widths a bit. For now, let's do an auto sweep from 2 GHz to 11 GHz with steps of 1 GHz.

We'll use our 44 Oe / 0.5 GHz and 63 Oe / 0.5 GHz figures to determine a field range for each frequency. We'll make a nice function that'll do that for us, provided the frequency and the width we want to leave out on eithr side of the centrepoint.

In [6]:
def freq_cp_3_3_5(frequency):
    m = (43.94472361809045) / (0.5)
    c = 106.4572864321608 - (3 * m)
    return m * frequency + c

def get_field_ranges(frequencies, width, num_points):
    field_ranges = []
    for frequency in frequencies:
        cp = freq_cp_3_3_5(frequency)
        field_range = np.linspace(cp - (width/2), cp + (width/2), num_points)
        field_ranges.append(field_range)
    return field_ranges

In [8]:
frequency_range = np.arange(6, 10 + 1, 1)
field_ranges = get_field_ranges(frequency_range, 300, 200)

Now we have a list of desired freqencies and their corresponding field ranges. It is time to get acquainted with `experiment.multisweep(primary_parameter=None, save_dir=None, fields=None, frequencies=None, closefig=True, savefig=False)`.

This function takes in:
- `primary_parameter` as either `frequency` or `field`. This indicates the parameter being held constant while the other is swept.
- `save_dir` a path to where you want to save your files.
- `frequencies` a list of frequencies or a list of frequency ranges (depending on what `primary_parameter` is)
- `fields` a list of fields or a list of field ranges (depending on what `primary_parameter` is)
- `closefig` a boolean to determine if you want to keep or close sweep plots as they finish (`True` recommended.)
- `savefig` a boolean determining if you want to save your sweep plots as they are completed.

We'll just pass in your frequencies along with their field ranges to `multisweep`.

In [9]:
save_dir = r'C:\Users\plyslab\Desktop\multisweep_2-11_ghz'
E.multisweep(primary_parameter='frequency', save_dir=save_dir, frequencies=frequency_range, fields=field_ranges,
             closefig=True, savefig=True)

c:\Users\plyslab\Desktop\SpinLab_FMR_Automation\experiment.py:215: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  self.ax.set_xlim(min(xdata), max(xdata))
c:\Users\plyslab\Desktop\SpinLab_FMR_Automation\experiment.py:215: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  self.ax.set_xlim(min(xdata), max(xdata))
c:\Users\plyslab\Desktop\SpinLab_FMR_Automation\experiment.py:215: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  self.ax.set_xlim(min(xdata), max(xdata))
c:\Users\plyslab\Desktop\SpinLab_FMR_Automation\experiment.py:215: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  self.ax.set_xlim(min(xdata), max(xdata))
c:\Users\plyslab\Desktop\SpinLab_FMR_Automation\experiment.py:215: UserWarning: Attempting to se

In [12]:
E.sweep_field(8.0, np.linspace(800, 950, 100), save_dir=save_dir, savefig=True)

c:\Users\plyslab\Desktop\SpinLab_FMR_Automation\experiment.py:215: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  self.ax.set_xlim(min(xdata), max(xdata))


In [5]:
save_dir = r'C:\Users\plyslab\Desktop\multisweep_2-11_ghz'
E.sweep_field(8.0, np.linspace(500, 1000, 100), save_dir=save_dir, savefig=True)

c:\Users\plyslab\Desktop\SpinLab_FMR_Automation\experiment.py:232: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  self.ax.set_xlim(min(xdata), max(xdata))


In [7]:
save_dir = r'C:\Users\plyslab\Desktop\multisweep_2-11_ghz'
E.sweep_field(7.0, np.linspace(400, 900, 300), save_dir=save_dir, savefig=True)

c:\Users\plyslab\Desktop\SpinLab_FMR_Automation\experiment.py:232: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  self.ax.set_xlim(min(xdata), max(xdata))


In [8]:
save_dir = r'C:\Users\plyslab\Desktop\multisweep_2-11_ghz'
E.sweep_field(2.0, np.linspace(0, 150, 300), save_dir=save_dir, savefig=True)

c:\Users\plyslab\Desktop\SpinLab_FMR_Automation\experiment.py:232: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  self.ax.set_xlim(min(xdata), max(xdata))


In [9]:
save_dir = r'C:\Users\plyslab\Desktop\multisweep_2-11_ghz'
E.sweep_field(4.0, np.linspace(50, 350, 300), save_dir=save_dir, savefig=True)

c:\Users\plyslab\Desktop\SpinLab_FMR_Automation\experiment.py:232: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  self.ax.set_xlim(min(xdata), max(xdata))


KeyboardInterrupt: 

Ni-FE Sample 2, 100nm

In [14]:
save_dir = r'C:\Users\plyslab\Desktop\ni_fe_2'
E.sweep_field(3.0, np.linspace(50, 200, 300), save_dir=save_dir, savefig=True)

c:\Users\plyslab\Desktop\SpinLab_FMR_Automation\experiment.py:232: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  self.ax.set_xlim(min(xdata), max(xdata))


In [15]:
save_dir = r'C:\Users\plyslab\Desktop\ni_fe_2'
E.sweep_field(8.0, np.linspace(500, 1000, 300), save_dir=save_dir, savefig=True)

c:\Users\plyslab\Desktop\SpinLab_FMR_Automation\experiment.py:232: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  self.ax.set_xlim(min(xdata), max(xdata))


In [5]:
save_dir = r'C:\Users\plyslab\Desktop\ni_fe_2'
E.sweep_field(14.0, np.linspace(1500, 2100, 200), save_dir=save_dir, savefig=True)

c:\Users\plyslab\Desktop\SpinLab_FMR_Automation\experiment.py:232: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  self.ax.set_xlim(min(xdata), max(xdata))


In [6]:
save_dir = r'C:\Users\plyslab\Desktop\ni_fe_2'
E.sweep_field(10.0, np.linspace(750, 1250, 200), save_dir=save_dir, savefig=True)

c:\Users\plyslab\Desktop\SpinLab_FMR_Automation\experiment.py:232: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  self.ax.set_xlim(min(xdata), max(xdata))


In [5]:
from srs_sr830_RS232 import SRS_SR830

In [6]:
lia = SRS_SR830(ResourceName='ASRL4::INSTR')

In [7]:
lia.query('*IDN?')

'Stanford_Research_Systems,SR830,s/n46107,ver1.07 '

In [9]:
lia.X()

5.28099e-05

In [12]:
foo = lia.X

In [13]:
foo()

3.83856e-05

In [1]:
round(1.433454545, 5)

1.43345